# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidYellowCurve2.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

# y = mx + b


def draw_lanes(image):
    image_copy_orig = np.copy(image)
    imshape = image.shape
    
    # seperate regions of interest
    #vertices_left = np.array([[ (100,imshape[0]), (imshape[1]/2-10, imshape[0]/2), (imshape[1]/2, imshape[0]/2),  (imshape[1]/2, imshape[0])]], dtype=np.int32)
    vertices_left = np.array([[(imshape[1]/2, imshape[0]/2), (imshape[1]/2-10, imshape[0]/2), (100, imshape[0]), (imshape[1]/2,imshape[0])]], dtype=np.int32)   
    vertices_right = np.array([[(imshape[1]/2, imshape[0]/2), (imshape[1]/2, imshape[0]), (imshape[1]-100,imshape[0]), (imshape[1]/2+10, imshape[0]/2)]], dtype=np.int32)

    # Idea is to try different values to detect a line 
    min_line_lengths = [80, 30]
    max_line_gaps = [80, 40]
    
    
    # detect left line
    b_left_avg, m_left_avg = None, None
    for max_line_gap in max_line_gaps:
        for min_length in min_line_lengths:
            try:
                [b_left_avg, m_left_avg] = detect_line(image, vertices_left, -0.9, -0.5,  min_length, max_line_gap)
                if np.isnan(m_left_avg):
                    continue
                else:
                    break
            except:
                pass
        
    # detect right line
    b_right_avg, m_right_avg = None, None
    for max_line_gap in max_line_gaps:
        for min_length in min_line_lengths:
            try:
                [b_right_avg, m_right_avg] = detect_line(image, vertices_right, 0.5, 0.9,  min_length, max_line_gap)
                if np.isnan(m_right_avg):
                    continue
                else:
                    break;
            except:
                pass
        

    # draw detected lines onto original image
    lines_detected = calculate_intersection(image, b_left_avg, m_left_avg, b_right_avg, m_right_avg) 
    draw_lines(image_copy_orig, lines_detected)
    
    return image_copy_orig




def detect_line(image, vertices, m_min, m_max,  min_line_length, max_line_gap):
    
    image_copy = np.copy(image)
 
    # remove darker colors
    frame_HSV = cv2.cvtColor(image_copy, cv2.COLOR_BGR2HSV)
    frame_threshold = cv2.inRange(frame_HSV, (0, 0, 210),(180, 255, 255))
    image_threshold = cv2.bitwise_and(image_copy, image_copy, mask=frame_threshold)
    plt.imshow(image_threshold)
#    plt.savefig("threshold.png")
    plt.figure()

    
    # to grayscale
    image_gray = grayscale(image_threshold)
    plt.imshow(image_gray, cmap='gray')
    plt.figure()
    
    # blur
    image_gaussian = gaussian_blur(image_gray, 5)

    # detect edges
    low_threshold = 150
    high_threshold = 200

    image_canny = canny(image_gaussian, low_threshold, high_threshold)
    plt.imshow(image_canny, cmap='gray')
#    plt.savefig("canny.png")
    plt.figure()

    # region of interest
    image_region = region_of_interest(image_canny, vertices)
    plt.imshow(image_region, cmap='gray')
    plt.figure()

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 1     # minimum number of votes (intersections in Hough grid cell)
    min_line_length_ = min_line_length #minimum number of pixels making up a line
    #max_line_gap = 80  # maximum gap in pixels between connectable line segments
    max_line_gap = max_line_gap  # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    [image_lines, lines] = hough_lines(image_region, rho, theta, threshold,
                                min_line_length_, max_line_gap)
    plt.imshow(image_lines, cmap='gray')
    plt.figure()


    # calculate m for each found line
    m = (lines[:,0][:,1] - lines[:,0][:,3]) / (lines[:,0][:,0] - lines[:,0][:,2])
    print("m: ", m)

    # slope has to be in certain range, filter out unrealistic slopes 
    filter_points = np.logical_and( m > m_min, m < m_max)
    print("m filter_points: ", filter_points)
    
    b = lines[:,0][:,1] - lines[:,0][:,0]*m
    print("b: ", b)
    
    points_avg = np.sum(lines[ filter_points], axis=0)/len(lines[filter_points])
    print("points_avg: ", points_avg)
    
    m_avg = (points_avg[0][1] - points_avg[0][3]) / (points_avg[0][0] - points_avg[0][2])
        
#    m_avg_from_m = np.sum(m[ filter_points], axis=0)/len(m[filter_points])   
#    print("diff m: ", m_avg_from_m - m_avg)
    print("m_avg: ", m_avg)

    b_avg = points_avg[0][1] - points_avg[0][0]*m_avg
    
#    b_avg_from_b = np.sum(b[filter_points], axis=0)/len(b[filter_points])   
#    print("diff b: ", b_avg_from_b - b_avg)  
    print("b_avg: ", b_avg)
    
    return b_avg, m_avg
    
    
def calculate_intersection(image, b_left_avg, m_left_avg, b_right_avg, m_right_avg)   : 

    # Calculate point where two lines intersect
    y_min = ((b_left_avg * m_right_avg) - (b_right_avg * m_left_avg)) / (m_right_avg - m_left_avg)
    print("y_min: ", y_min)

    x_min = (b_right_avg - b_left_avg) / (m_left_avg - m_right_avg)
    print("x_min: ", x_min)

    y_max = image.shape[0]   #max y
    x_left = (y_max - b_left_avg) / m_left_avg #- 20 , using m[4] gives better result why?
    x_right = (y_max - b_right_avg) / m_right_avg #- 20 , using m[4] gives better result why?

    
    line_left = [int(round(x_left)), y_max, int(round(x_min)), int(round(y_min))]
    line_right = [int(round(x_min)), int(round(y_min)), int(round(x_right)), y_max]
    print("line_left: ", line_left)
    print("line_right: ", line_right)

    lines_detected = np.array([[line_left],[line_right]])
    print("lines_detected: ", lines_detected)
    
    return lines_detected
    


In [ ]:
plt.imshow(draw_lanes(image), cmap='gray')
#plt.savefig("detected.png")
plt.figure()

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = detect_line(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(4,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(draw_lanes) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

m:  [-0.72670807 -0.83536585 -0.92473118 -0.72093023 -0.75083056 -0.83870968
 -1.43333333]
m filter_points:  [ True  True False  True  True  True False]
b:  [666.42857143 705.2195122  721.17204301 657.04651163 672.3986711
 705.16129032 921.6       ]
points_avg:  [[259.8 480.6 439.4 341. ]]
m_avg:  -0.7772828507795103
b_avg:  682.5380846325168
m:  [-0.73333333 -0.81132075 -0.75       -0.81521739 -0.78571429 -0.80612245]
m filter_points:  [ True  True  True  True  True  True]
b:  [667.8        697.83018868 669.5        699.59782609 686.78571429
 695.64285714]
points_avg:  [[255.16666667 487.16666667 352.83333333 410.66666667]]
m_avg:  -0.7832764505119455
b_avg:  687.0327076222981
m:  [0.53676471 0.55555556 0.58823529 0.55294118 0.53051643 0.53719008
 0.53398058 0.56034483]
m filter_points:  [ True  True  True  True  True  True  True  True]
b:  [54.69852941 39.44444444 24.05882353 40.30588235 57.35211268 55.71900826
 58.98058252 37.15517241]
points_avg:  [[586.5   368.625 709.625 435.875]

t:  89%|██████████████████████████████████████████████████████████▌       | 196/221 [05:27<00:43,  1.73s/it, now=None]

m:  [-0.70212766 -0.7804878  -0.80833333 -0.84166667 -0.84       -0.81410256
 -0.72868217]
m filter_points:  [ True  True  True  True  True  True  True]
b:  [663.68085106 685.85365854 698.38333333 706.95       704.48
 695.87820513 662.76744186]
points_avg:  [[284.         463.71428571 441.         338.57142857]]
m_avg:  -0.797088262056415
b_avg:  690.0873521383076
m:  [-0.78225806 -0.77380952 -0.75396825]
m filter_points:  [ True  True  True]
b:  [686.7016129  677.51190476 671.64285714]
points_avg:  [[295.         451.66666667 406.33333333 366.        ]]
m_avg:  -0.7694610778443117
b_avg:  678.6576846307387
m:  [0.62051282 0.55440415 0.5625     0.61842105 0.69879518 0.53787879
 0.96296296 0.56       0.53125   ]
m filter_points:  [ True  True  True  True  True  True False  True  True]
b:  [   2.44615385   43.1761658    40.25          4.86842105  -62.80722892
   53.39393939 -210.7037037    37.76         59.84375   ]
points_avg:  [[579.75  364.375 710.625 440.875]]
m_avg:  0.5845272206303

t:  89%|██████████████████████████████████████████████████████████▊       | 197/221 [05:29<00:41,  1.73s/it, now=None]

m:  [-0.80597015 -0.75362319 -0.8358209  -0.84444444 -0.75177305]
m filter_points:  [ True  True  True  True  True]
b:  [695.44776119 673.73913043 702.34328358 704.73333333 671.09929078]
points_avg:  [[348.6 410.8 476.  309.6]]
m_avg:  -0.794348508634223
b_avg:  687.7098901098902
m:  [-0.34210526 -0.17948718 -3.54545455 -0.82857143 -0.97058824 -0.74
 -0.04       -0.90789474 -0.68292683  0.02777778 -1.         -0.90196078
 -2.3125     -0.97058824 -0.81578947]
m filter_points:  [False False False  True False  True False False  True False False False
 False False  True]
b:  [ 466.81578947  396.61538462 1869.          701.85714286  730.02941176
  670.46        324.92        736.43421053  641.3902439   299.66666667
  764.          728.37254902 1336.75        728.08823529  697.78947368]
points_avg:  [[244.75 494.   285.75 462.75]]
m_avg:  -0.7621951219512195
b_avg:  680.547256097561
m:  [0.60082305 0.57416268 0.57647059 0.60240964 0.6       ]
m filter_points:  [ True  True  True  True  True]

...

t: 100%|█████████████████████████████████████████████████████████████████▋| 220/221 [06:10<00:02,  2.02s/it, now=None]

m:  [-0.81327801 -0.77647059 -0.77710843 -0.7804878  -0.8089172  -1.0326087
 -0.81147541]
m filter_points:  [ True  True  True  True  True False  True]
b:  [696.58921162 675.43529412 679.26506024 675.70731707 693.4522293
 775.60869565 691.72131148]
points_avg:  [[227.66666667 504.16666667 369.83333333 390.66666667]]
m_avg:  -0.7983587338804221
b_avg:  685.9263384134427
m:  [-0.8125     -0.77647059 -0.75294118 -0.7804878  -0.75      ]
m filter_points:  [ True  True  True  True  True]
b:  [696.4375     675.43529412 672.28235294 675.70731707 672.        ]
points_avg:  [[250.8 485.6 334.  421.2]]
m_avg:  -0.774038461538462
b_avg:  679.7288461538462
m:  [0.58       0.53488372 0.58163265 0.53636364]
m filter_points:  [ True  True  True  True]
b:  [29.38       52.69767442 29.         53.24545455]
points_avg:  [[555.75 351.5  703.75 435.25]]
m_avg:  0.5658783783783784
b_avg:  37.013091216216196
m:  [0.58       0.53488372 0.58163265 0.53636364]
m filter_points:  [ True  True  True  True]
b:  [2

t: 100%|██████████████████████████████████████████████████████████████████| 221/221 [06:12<00:00,  1.92s/it, now=None]

m:  [-0.80851064 -0.81111111]
m filter_points:  [ True  True]
b:  [693.89361702 693.77777778]
points_avg:  [[340.  418.5 432.  344. ]]
m_avg:  -0.8097826086956522
b_avg:  693.8260869565217
m:  [-0.80851064 -0.81111111]
m filter_points:  [ True  True]
b:  [693.89361702 693.77777778]
points_avg:  [[340.  418.5 432.  344. ]]
m_avg:  -0.8097826086956522
b_avg:  693.8260869565217
m:  [0.57317073 0.57731959 0.5785124  0.58045977 0.57718121]
m filter_points:  [ True  True  True  True  True]
b:  [29.48780488 30.40206186 30.18181818 29.85632184 26.3557047 ]
points_avg:  [[558.6 351.8 723.8 447.2]]
m_avg:  0.5774818401937047
b_avg:  29.218644067796504
m:  [0.57317073 0.57731959 0.5785124  0.58045977 0.57718121]
m filter_points:  [ True  True  True  True  True]
b:  [29.48780488 30.40206186 30.18181818 29.85632184 26.3557047 ]
points_avg:  [[558.6 351.8 723.8 447.2]]
m_avg:  0.5774818401937047
b_avg:  29.218644067796504
y_min:  305.87730812646436
x_min:  479.07768660893
line_left:  [190, 540, 479,

Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 6min 14s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(3,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(draw_lanes)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(4,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')

challenge_clip = clip3.fl_image(draw_lanes)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))